读取数据，并划分数据集，验证集以及测试集

In [1]:
import gc
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

matrix = pd.read_csv('./result/matrix.csv')
matrix = matrix.drop(matrix.columns[0],axis=1)

#train、test-setdata
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']

#导入分析库
from sklearn.model_selection import train_test_split
# X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.3)
# 最终版提交，使用所有训练数据，训练模型
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=0.1)
X_train, X_valid, y_train, y_valid = X_train.values, X_valid.values, y_train.values.astype('float32'), y_valid.values.astype('float32') 
test_data = test_data.values

gc.collect()
# del data_train,temp

10

### 单模型训练

将训练集划分为训练与测试集合，在各种模型上进行测试。

评价指标有精度，回召率，准确度，混淆矩阵，AUC面积以及分类报告

如下，SVM，GMM，DT表现效果一般，舍弃。另外KNN时间代价太大，舍弃。

In [2]:
import time
import xgboost as xgb
from model import model_train
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

model_dict = {'svm':SVC,'logistic':LogisticRegression,'decision_tree':DecisionTreeClassifier,'bayes':GaussianNB,
               'mixture':GaussianMixture,'KNN':KNeighborsClassifier,'GBDT':GradientBoostingClassifier,'XGBoost':XGBClassifier}
# svm与mixture效果都不好，knn模型则训练时间过程
# model_train('svm',model=model_dict['svm'])
# model_train('KNN',model=model_dict['KNN'])
# model_train('mixture',model=model_dict['mixture'],x_train=X_train,y_train=y_train,x_val=X_valid,y_val=y_valid)

# model = model_train('logistic',model=model_dict['logistic'],x_train=X_train,y_train=y_train,x_val=X_valid,y_val=y_valid)
# model = model_train('bayes',model=model_dict['bayes'],x_train=X_train,y_train=y_train,x_val=X_valid,y_val=y_valid)
# model = model_train('decision_tree',model=model_dict['decision_tree'],x_train=X_train,y_train=y_train,x_val=X_valid,y_val=y_valid)
# model = model_train('GBDT',model=model_dict['GBDT'],x_train=X_train,y_train=y_train,x_val=X_valid,y_val=y_valid)
model = model_train('XGBoost',model=model_dict['XGBoost'],x_train=X_train,y_train=y_train,x_val=X_valid,y_val=y_valid)


模型GBDT:
模型GBDT, 耗时 2.5543150345484418 minutes:
使用clf.score计算, 测试集的准确度为0.9397017671637214
测试集的ROC面积为0.7218786925565353
测试集的混淆矩阵为:
[[24513     8]
 [ 1565     1]]
测试集的分类报告为:
              precision    recall  f1-score   support

         0.0       0.94      1.00      0.97     24521
         1.0       0.11      0.00      0.00      1566

    accuracy                           0.94     26087
   macro avg       0.53      0.50      0.49     26087
weighted avg       0.89      0.94      0.91     26087



生成提交文件

In [ ]:
paths = './data_format1'
submission = pd.read_csv(f'{paths}/test_format1.csv')

prob = model(test_data)[:,1]
submission['prob'] = prob
submission.to_csv('./result/submission.csv', index=False)

由于正负样本比例相差较大，使用了8个模型训练效果都较差。正类的召回率与精度都很低

解决方案见模型优化